<a href="https://colab.research.google.com/github/hannapalya/anomaly_detection_syndromic/blob/main/OCSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
One-Class SVM - CPU Standalone (Expanded HP Search)
"""

import os, numpy as np, pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from typing import List, Dict, Tuple

# ===== CONFIG =====
DATA_DIR = "/content"
SIGNALS = list(range(1, 17))
DAYS_PER_YEAR = 364
TRAIN_YEARS = 6
TRAIN_DAYS = TRAIN_YEARS * DAYS_PER_YEAR
VALID_DAYS = 49 * 7
RNG_STATE = 42

SPEC_TARGET = 0.95
W_SENS, W_SPEC = 2.0, 3.0

# Expanded grid for CPU (9 configs)
HP_GRID = [
    (7,  0.03), (7,  0.05), (7,  0.07),
    (14, 0.03), (14, 0.05), (14, 0.07),
    (21, 0.03), (21, 0.05), (21, 0.07),
]

MAX_TRAIN = 50000

print("=" * 60)
print("ONE-CLASS SVM (CPU) - Expanded HP Search")
print(f"Testing {len(HP_GRID)} configurations per signal")
print("=" * 60)

# ===== HELPERS =====
def load_data(sig):
    X = pd.read_csv(os.path.join(DATA_DIR, f"simulated_totals_sig{sig}.csv"))
    Y = (pd.read_csv(os.path.join(DATA_DIR, f"simulated_outbreaks_sig{sig}.csv")) > 0).astype(int)
    date_col = next((c for c in ["date", "Date", "ds", "timestamp"] if c in X.columns), None)
    if date_col:
        X = X.drop(columns=[date_col])
        if date_col in Y.columns: Y = Y.drop(columns=[date_col])
    return X, Y

def cross_sim_split(sims: List[dict], rng: np.random.RandomState, train_frac=0.6):
    rng.shuffle(sims)
    n_train = int(len(sims) * train_frac)
    return sims[:n_train], sims[n_train:]

def create_features(series, window_size):
    features = []
    for i in range(window_size-1, len(series)):
        w = series[max(0, i-window_size+1):i+1]
        cur = series[i]
        mean_w = np.mean(w)
        std_w = np.std(w)
        feats = [cur, mean_w, std_w, cur/(mean_w+1e-6)]
        features.append(feats)
    return np.array(features, dtype=np.float32)

def sens_spec(y_true, y_pred):
    if len(y_true) == 0: return 0.0, 0.0
    TN, FP, FN, TP = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return (TP/(TP+FN) if (TP+FN)>0 else 0.0, TN/(TN+FP) if (TN+FP)>0 else 0.0)

def tune_threshold(y_val, scores):
    best_t, best_score = None, -1.0
    for p in [1, 2, 3, 5, 7, 10]:
        thr = np.percentile(scores, p)
        s, sp = sens_spec(y_val, (scores <= thr).astype(int))
        if sp >= SPEC_TARGET:
            score = W_SENS*s + W_SPEC*sp
            if score > best_score: best_score, best_t = score, float(thr)
    if best_t is None: best_t = float(np.percentile(scores, 2.0))
    return best_t

def _align_tail(O, T):
    return O[-T:] if len(O) >= T else np.pad(O, (T-len(O), 0))

def metric_sensitivity(A, O):
    Oa = _align_tail(O, A.shape[0])
    TP = np.logical_and(A==1, Oa>0).sum()
    FN = np.logical_and(A==0, Oa>0).sum()
    return (TP/(TP+FN)) if (TP+FN)>0 else np.nan

def metric_specificity(A, O):
    Oa = _align_tail(O, A.shape[0])
    TN = np.logical_and(A==0, Oa==0).sum()
    FP = np.logical_and(A==1, Oa==0).sum()
    return (TN/(TN+FP)) if (TN+FP)>0 else np.nan

def metric_pod(A, O):
    Oa = _align_tail(O, A.shape[0])
    return np.mean((np.logical_and(A==1, Oa>0)).sum(axis=0) > 0)

def metric_timeliness(A, O):
    Oa = _align_tail(O, A.shape[0])
    T, J = A.shape
    score = 0.0
    for j in range(J):
        y, a = Oa[:,j], A[:,j]
        idx_out = np.where(y>0)[0]
        if len(idx_out)==0: score += 1.0; continue
        idx_hit = np.where((a==1)&(y>0))[0]
        if len(idx_hit)==0: score += 1.0; continue
        r1, r2 = int(idx_out[0]), int(idx_out[-1])
        obs = int(idx_hit[0])
        score += (obs - r1) / (r2 - r1 + 1)
    return score / J

# ===== MAIN =====
np.random.seed(RNG_STATE)
rng = np.random.RandomState(RNG_STATE)

summary = {}

for S in SIGNALS:
    print(f"\n{'='*60}")
    print(f"Signal {S}")
    print(f"{'='*60}")

    import time
    signal_start = time.time()

    try:
        Xsig, Ysig = load_data(S)
    except FileNotFoundError:
        print(f"Files not found for signal {S}; skipping.")
        continue

    sims: List[dict] = []
    for sim_idx, col in enumerate(Xsig.columns):
        x = Xsig[col].to_numpy(float)
        y = Ysig[col].to_numpy(int)
        if len(x) >= TRAIN_DAYS + VALID_DAYS:
            sims.append(dict(sim=f"sig{S}_sim{sim_idx}", x=x, y=y))

    if not sims:
        print("No complete sims; skip.")
        continue

    train_sims, held_sims = cross_sim_split(sims, rng, train_frac=0.6)
    mid = max(1, len(held_sims)//2)
    val_sims = held_sims[:mid]
    test_sims = held_sims[mid:] if len(held_sims) > 1 else held_sims

    print(f"Train: {len(train_sims)}, Val: {len(val_sims)}, Test: {len(test_sims)}")

    # Hyperparam search
    best = dict(score=-1.0, params=None)

    for idx, (WIN, NU) in enumerate(HP_GRID):
        print(f"  Config {idx+1}/{len(HP_GRID)}: WIN={WIN}, NU={NU}...", end='', flush=True)

        # Train features
        Xtr_list = []
        for d in train_sims:
            feats = create_features(d["x"][:TRAIN_DAYS], WIN)
            if len(feats): Xtr_list.append(feats)

        if not Xtr_list:
            print(" no train data")
            continue

        Xtr = np.concatenate(Xtr_list)
        if len(Xtr) > MAX_TRAIN:
            Xtr = Xtr[rng.choice(len(Xtr), MAX_TRAIN, replace=False)]

        scaler = StandardScaler().fit(Xtr)
        Xtr_s = scaler.transform(Xtr)

        # Val features
        Xv_list, Yv_list = [], []
        for d in val_sims:
            x_tail = d["x"][-VALID_DAYS:]
            y_tail = d["y"][-VALID_DAYS:]
            feats = create_features(x_tail, WIN)
            if len(feats):
                Xv_list.append(feats)
                Yv_list.append(y_tail[WIN-1:])

        if not Xv_list:
            print(" no val data")
            continue

        Xval = np.concatenate(Xv_list)
        Yval = np.concatenate(Yv_list)
        Xval_s = scaler.transform(Xval)

        # Fit model
        svm = OneClassSVM(kernel='rbf', gamma='scale', nu=NU, cache_size=200, max_iter=1000)
        svm.fit(Xtr_s)

        # Validate
        val_scores = svm.decision_function(Xval_s)
        thr = tune_threshold(Yval, val_scores)
        yhat = (val_scores <= thr).astype(int)
        s, sp = sens_spec(Yval, yhat)

        score = (W_SENS*s + W_SPEC*sp) if sp >= SPEC_TARGET else sp

        print(f" s={s:.2f} sp={sp:.2f}")

        if score > best["score"]:
            best.update(score=score, params={'WIN':WIN, 'NU':NU, 'scaler':scaler, 'svm':svm}, threshold=thr)

    if best["params"] is None:
        print("No valid HP config found.")
        continue

    P = best["params"]
    print(f"✓ Best: WIN={P['WIN']}, NU={P['NU']}, thr={best['threshold']:.6f}")

    # Test
    per_sim_preds, per_sim_labels = [], []

    for d in test_sims:
        x_tail = d["x"][-VALID_DAYS:]
        y_tail = d["y"][-VALID_DAYS:]
        feats = create_features(x_tail, P['WIN'])

        if len(feats):
            feats_s = P['scaler'].transform(feats)
            scores = P['svm'].decision_function(feats_s)
            preds = (scores <= best['threshold']).astype(int)
            per_sim_preds.append(preds)
            per_sim_labels.append(y_tail[P['WIN']-1:])

    if not per_sim_preds:
        print("No test features.")
        continue

    A = np.stack(per_sim_preds, axis=1)
    O = np.stack(per_sim_labels, axis=1)

    sens = metric_sensitivity(A, O)
    spec = metric_specificity(A, O)
    pod = metric_pod(A, O)
    tim = metric_timeliness(A, O)

    elapsed = time.time() - signal_start
    print(f"TEST → Sens={sens:.3f}, Spec={spec:.3f}, POD={pod:.3f}, Tim={tim:.3f} ({elapsed:.1f}s)")

    summary[S] = dict(
        Sensitivity_All=sens, Specificity_All=spec,
        POD_All=pod, Timeliness_All=tim,
        Threshold=best['threshold'], Nu=P['NU'], Win=P['WIN']
    )

# Save
if summary:
    df = pd.DataFrame.from_dict(summary, orient="index")
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    print(df)
    print("\nMeans:")
    print(df[['Sensitivity_All','Specificity_All','POD_All','Timeliness_All']].mean())
    df.to_csv("OneClassSVM_CPU_results.csv")
    print("\nSaved: OneClassSVM_CPU_results.csv")
else:
    print("\nNo results to save.")

ONE-CLASS SVM (CPU) - Expanded HP Search
Testing 9 configurations per signal

Signal 1
Train: 60, Val: 20, Test: 20
  Config 1/9: WIN=7, NU=0.03...

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


 s=0.29 sp=0.96
  Config 2/9: WIN=7, NU=0.05...

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


 s=0.23 sp=0.98
  Config 3/9: WIN=7, NU=0.07...

...

 s=0.75 sp=0.97
  Config 8/9: WIN=21, NU=0.05...

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


 s=0.75 sp=0.97
  Config 9/9: WIN=21, NU=0.07...

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


 s=0.75 sp=0.97
✓ Best: WIN=14, NU=0.07, thr=-525.780436
TEST → Sens=0.799, Spec=0.961, POD=1.000, Tim=0.114 (82.2s)

SUMMARY
    Sensitivity_All  Specificity_All  POD_All  Timeliness_All   Threshold  \
1          0.300310         0.959950     0.95        0.180110  -39.451421   
2          0.396450         0.956150     0.85        0.254808    4.159671   
3          0.802589         0.959148     1.00        0.130093 -438.769652   
4          0.907407         0.948935     1.00        0.078322  -38.345641   
5          0.342767         0.950505     0.95        0.168674 -291.441889   
6          0.337079         0.960204     0.70        0.404951   29.629815   
7          0.845912         0.967367     1.00        0.071144 -328.353820   
8          0.264840         0.974455     0.60        0.463117    1.537241   
9          0.770642         0.953776     1.00        0.104644  -94.313396   
10         0.257627         0.979381     0.70        0.446136  -34.599159   
11         0.600000        